In [16]:
import image_slicer
import cv2
import numpy as np
import os

In [122]:
for file in os.listdir('.\\raw'):
    separate_file(os.path.join('.\\raw', file))

In [120]:
np.set_printoptions(threshold=np.inf)


def separate_file(file_name):
    if not os.path.exists('.\\temp'): 
        os.mkdir('.\\temp')
    
    #clear outer white border
    original = cv2.imread(file_name, 0)
    mask = original > 30
    
    mask_left = mask[:, 0:mask.shape[1]//2]
    left = np.max(np.append(np.where(mask_left.all(axis=0))[0], 0))
    
    mask_right = mask[:, mask.shape[1]//2:mask.shape[1]]
    right = np.min(np.append(np.where(mask_right.all(axis=0))[0], mask.shape[1]//2)) + mask.shape[1]//2
    
    mask_top = mask[0:mask.shape[0]//2, :]
    top = np.max(np.append(np.where(mask_top.all(axis=1))[0], 0))
    
    mask_bottom = mask[mask.shape[0]//2:mask.shape[0], :]
    bottom = np.min(np.append(np.where(mask_bottom.all(axis=1))[0], mask.shape[0]//2)) + mask.shape[0]//2
    
    if abs(left - right) < original.shape[1] * .1 or abs(top - bottom) < original.shape[0] * .1:
        cv2.imwrite(os.path.join('.\\temp', 'original.jpg'), original)
    else:
        cv2.imwrite(os.path.join('.\\temp', 'original.jpg'), original[top:bottom, left:right])

    #slice into singles
    tiles = image_slicer.slice(os.path.join('.\\temp', 'original.jpg'), 81, save=False)
    image_slicer.save_tiles(tiles, directory='.\\temp', prefix='temp')
    
    #box around original
    for i in range(1, 10):
        for j in range(1, 10):
            file = cv2.imread(os.path.join('.\\temp', f'temp_0{j}_0{i}.png'), 0)
            mask = file < 235
            
            mask_left = mask[:, 0:mask.shape[1]//2]
            left = np.max(np.append(np.where(mask_left.all(axis=0))[0], 0))
            
            mask_right = mask[:, mask.shape[1]//2:mask.shape[1]]
            right = np.min(np.append(np.where(mask_right.all(axis=0))[0], mask.shape[1]//2)) + mask.shape[1]//2
            
            mask_top = mask[0:mask.shape[0]//2, :]
            top = np.max(np.append(np.where(mask_top.all(axis=1))[0], 0))
            
            mask_bottom = mask[mask.shape[0]//2:mask.shape[0], :]
            bottom = np.min(np.append(np.where(mask_bottom.all(axis=1))[0], mask.shape[0]//2)) + mask.shape[0]//2
            
            best = max([left, top, file.shape[0]-bottom, file.shape[1]-right])       
            file = file[best+1:file.shape[0]-best-1, best+1:file.shape[1]-best-1]
            
            mask = file < 200
            coords = np.argwhere(mask)
            if len(coords) == 0:
                cv2.imwrite(os.path.join(".\\output", f'{file_name[6:8]}{j}{i}.jpg'), cv2.resize(file, (28, 28)))
            else:
                x0, y0 = coords.min(axis=0)
                x1, y1 = coords.max(axis=0) + 1
                cv2.imwrite(os.path.join(".\\output", f'{file_name[6:8]}{j}{i}.jpg'), cv2.resize(file[x0:x1, y0:y1], (28, 28)))
        
    #delete temp files
    for file in os.listdir('.\\temp'):
        os.unlink(os.path.join('.\\temp', file))
    os.rmdir('.\\temp')